In [1]:
import cPickle as pickle
import gzip
import numpy
from midi_to_statematrix import *

import multi_training
import model
from RBM import rbm_train, rbm_reconstruct, rbm_reconstruction
from main import gen_adaptive

Using gpu device 0: GeForce GTX 860M (CNMeM is enabled with initial size: 80.0% of memory, CuDNN 4004)


In [2]:
#pieces_major = multi_training.loadPieces('music/major')
#pieces_minor = multi_training.loadPieces('music/minor')
path = 'music'
pieces = multi_training.loadPieces(dirpath = path)

Loaded preprocessed notes...


In [3]:
m = model.Model([300, 300], [100, 50], dropout=0.8)

model.py:367: UserWarning: Updating an `OrderedUpdates` with a non-ordered dictionary with 2+ elements could make your code non-deterministic
  self.walk_input: next_input
model.py:370: UserWarning: Updating an `OrderedUpdates` with a non-ordered dictionary with 2+ elements could make your code non-deterministic
  updates.update({hidden:newstate for hidden, newstate, layer in zip(self.walk_hiddens, new_states, self.time_model.layers) if has_hidden(layer)})


In [4]:
m.learned_config = pickle.load(open("output/params11000.p", "rb"))

In [4]:
batches = 3000
batches_old = 900

print('Training {0}+{1} batches on {2}'.format(batches,batches_old, path))

multi_training.trainPiece(m, pieces, [batches, batches_old])#, notes_to_input = None)
# add starting output names from given batch
pickle.dump(m.learned_config, open("output/final_learned_config_{0}.p".format(1*batches), "wb"))

Training 1000+0 batches on music
epoch 0, error=102413.179688
epoch 100, error=16254.8369141
epoch 200, error=14838.8476562
epoch 300, error=17746.5039062
epoch 400, error=16246.8847656
epoch 500, error=13889.5576172
epoch 600, error=14049.15625
epoch 700, error=15409.0791016
epoch 800, error=12983.3642578
epoch 900, error=13032.5244141


In [5]:
batches = 3000
batches_old = 900

print('Training {0}+{1} batches on {2}'.format(batches,batches_old, path))

multi_training.trainPiece(m, pieces, [batches, batches_old])#, notes_to_input = None)
# add starting output names from given batch
pickle.dump(m.learned_config, open("output/final_learned_config_{0}.p".format(1*batches), "wb"))

Training 3000+900 batches on music
epoch 900, error=14043.7988281
epoch 1000, error=14774.8447266
epoch 1100, error=10056.7998047
epoch 1200, error=8943.20996094
epoch 1300, error=7917.62988281
epoch 1400, error=8261.27441406
epoch 1500, error=7652.38867188


In [63]:
gen_adaptive(m,dict(pieces.items()),5,name="composition_11000_4:46", rbm=rbm)

(99996,)
RBM reconstruction: 0/97500. Time (m:s) - 57:19
(99996,)
RBM reconstruction: 156/97500. Time (m:s) - 57:20
(99996,)
RBM reconstruction: 312/97500. Time (m:s) - 57:21
(99996,)
RBM reconstruction: 468/97500. Time (m:s) - 57:22
(99996,)
RBM reconstruction: 624/97500. Time (m:s) - 57:23
(99996,)
RBM reconstruction: 780/97500. Time (m:s) - 57:25
(99996,)
RBM reconstruction: 936/97500. Time (m:s) - 57:26
(99996,)
RBM reconstruction: 1092/97500. Time (m:s) - 57:27
(99996,)
RBM reconstruction: 1248/97500. Time (m:s) - 57:28
(99996,)
RBM reconstruction: 1404/97500. Time (m:s) - 57:29
(99996,)
RBM reconstruction: 1560/97500. Time (m:s) - 57:30
(99996,)
RBM reconstruction: 1716/97500. Time (m:s) - 57:32
(99996,)
RBM reconstruction: 1872/97500. Time (m:s) - 57:33
(99996,)
RBM reconstruction: 2028/97500. Time (m:s) - 57:34
(99996,)
RBM reconstruction: 2184/97500. Time (m:s) - 57:35
(99996,)
RBM reconstruction: 2340/97500. Time (m:s) - 57:36
(99996,)
RBM reconstruction: 2496/97500. Time (m:

In [59]:
pickle.dump(rbm.W, open("output/rbm.W.pkl", "wb"))
pickle.dump(rbm.vbias, open("output/rbm.vbias.pkl", "wb"))

In [85]:
#%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [5]:
rbm = rbm_train(np.array(dict(pieces.items()[:]).values()), window_len = 16, learning_rate=0.1,
                training_epochs=1, batch_size=20, n_hidden=1500)

         +25 files trained on, time (m:s) - 5:45
         +25 files trained on, time (m:s) - 11:11
         +25 files trained on, time (m:s) - 15:19
         +25 files trained on, time (m:s) - 19:12
         +25 files trained on, time (m:s) - 22:16
         +25 files trained on, time (m:s) - 25:27
         +25 files trained on, time (m:s) - 28:40
         +25 files trained on, time (m:s) - 33:3
         +25 files trained on, time (m:s) - 37:56
         +25 files trained on, time (m:s) - 42:30
         +25 files trained on, time (m:s) - 49:47
Training epoch 0, cost is -128.335205078, time (m:s) - 52:44
Training took 47.121287 minutes


In [47]:
from __future__ import print_function

from datetime import datetime

import numpy as np
import six.moves.cPickle as pickle
import theano
import theano.tensor as T
import os
from theano.tensor.shared_randomstreams import RandomStreams
from RBM import share_data, RBM

train_data = np.array(pieces.values())
window_len = 16
learning_rate=0.1
batch_size=20
n_hidden=1500

# allocate symbolic variables for the data
index = T.lscalar()
x = T.matrix('x')  # the data is presented as rasterized images
beat_stride = 78*2
rng = np.random.RandomState(123)
theano_rng = RandomStreams(rng.randint(2 ** 30))

# initialize storage for the persistent chain (state = hidden layer of chain)
persistent_chain = theano.shared(np.zeros((batch_size, n_hidden), dtype=theano.config.floatX), borrow=True)

# construct the RBM class
rbm = RBM(input=x, n_visible=window_len*beat_stride,
          n_hidden=n_hidden, numpy_rng=rng, theano_rng=theano_rng)

# get the cost and the gradient corresponding to one step of CD-15
cost, updates = rbm.get_cost_updates(lr=learning_rate,
                                     persistent=persistent_chain,
                                     k=15)

#################################
#     Training the RBM          #
#################################

# start-snippet-5
# it is ok for a theano function to have no output
# the purpose of train_rbm is solely to update the RBM parameters


def share_train(data):
    shared_ = share_data(make_windows(data))
    return shared_

def make_windows(data):
    data = data.reshape((-1, 78 * 2))
    data = np.array([data[idx:idx + window_len] for idx in range(len(data) - window_len)])
    data = data.reshape((-1, 16 * 78 * 2))
    return data.astype('float32')

train_set = share_train(train_data[0])

train_rbm = theano.function(
    [index],
    cost,
    updates=updates,
    givens={
        x: train_set[index:index+batch_size]
    },
    name='train_rbm'
)

def train(training_epochs):
    # go through training epochs
    for epoch in range(training_epochs):
        # go through the training set
        mean_cost = []
        for t in range(0,len(train_data)):
            #tm = datetime.now()
            train_set.set_value(make_windows(train_data[t]))
            #print('Time wasted on data compilation :', datetime.now()-tm)
            idxs = train_data[t].shape[0]
            for idx in range(0,idxs-batch_size,batch_size):
                mean_cost += [train_rbm(idx)]
                #idx = random.choice(range(len(train_set)))
                #piece_output = train_set.values()[idx]
                #start = random.randrange(0,len(piece_output)-window_len,division_len)
            if t % 25 == 0:
                tm = datetime.now()
                print('         +25 files trained on, time (m:s) - {0}:{1}'.format(tm.minute, tm.second))
        tm = datetime.now()
        print('Training epoch {0}, cost is {1}, time (m:s) - {2}:{3}'.format(epoch, np.mean(mean_cost), tm.minute, tm.second))

In [48]:
training_epochs = 1
train(training_epochs)

         +25 files trained on, time (m:s) - 44:54
         +25 files trained on, time (m:s) - 49:45
         +25 files trained on, time (m:s) - 53:59
         +25 files trained on, time (m:s) - 57:39
         +25 files trained on, time (m:s) - 1:39
         +25 files trained on, time (m:s) - 6:6
         +25 files trained on, time (m:s) - 9:59
         +25 files trained on, time (m:s) - 14:29
         +25 files trained on, time (m:s) - 18:5
         +25 files trained on, time (m:s) - 22:2
         +25 files trained on, time (m:s) - 32:54
Training epoch 0, cost is -116.842918396, time (m:s) - 39:13


In [50]:
train(training_epochs)

         +25 files trained on, time (m:s) - 43:59
         +25 files trained on, time (m:s) - 48:44
         +25 files trained on, time (m:s) - 52:44
         +25 files trained on, time (m:s) - 56:12
         +25 files trained on, time (m:s) - 59:47
         +25 files trained on, time (m:s) - 3:30
         +25 files trained on, time (m:s) - 6:59
         +25 files trained on, time (m:s) - 11:28
         +25 files trained on, time (m:s) - 15:3
         +25 files trained on, time (m:s) - 18:42
         +25 files trained on, time (m:s) - 26:19
Training epoch 0, cost is -86.5165634155, time (m:s) - 30:25


In [54]:
train(15)
#reduce gibbs step, so that it doesn't converge too much into single pattern

         +25 files trained on, time (m:s) - 0:10
         +25 files trained on, time (m:s) - 5:36
         +25 files trained on, time (m:s) - 10:30
         +25 files trained on, time (m:s) - 14:23
         +25 files trained on, time (m:s) - 18:23
         +25 files trained on, time (m:s) - 22:40
         +25 files trained on, time (m:s) - 26:40
         +25 files trained on, time (m:s) - 31:36
         +25 files trained on, time (m:s) - 35:13
         +25 files trained on, time (m:s) - 39:0
         +25 files trained on, time (m:s) - 46:35
Training epoch 0, cost is -74.0345230103, time (m:s) - 50:41
         +25 files trained on, time (m:s) - 50:47
         +25 files trained on, time (m:s) - 55:30
         +25 files trained on, time (m:s) - 59:24
         +25 files trained on, time (m:s) - 2:50
         +25 files trained on, time (m:s) - 6:20
         +25 files trained on, time (m:s) - 10:0
         +25 files trained on, time (m:s) - 13:28
         +25 files trained on, time (m:s) - 1

KeyboardInterrupt: 

In [80]:
def reconstruct_multiple(dirpath='reconstruction', pieces_fname ='/pieces.pkl'):
    pieces = {}
    pieces_fname = dirpath+pieces_fname
    ignores = 0
    for fname in os.listdir(dirpath):
        if fname.lower()[-4:] != '.mid' or 'rbm' in fname.lower():
            continue
        outMatrix = midiToNoteStateMatrix(os.path.join(dirpath, fname))
        rbm_outputs, rbm_outputs_discrete = rbm_reconstruct(rbm, np.array(outMatrix), window_len = 16)

        noteStateMatrixToMidi(np.array(rbm_outputs), 'reconstruction/' + fname + ' rbm')
        noteStateMatrixToMidi(np.array(rbm_outputs_discrete), 'reconstruction/' + fname + ' discrete rbm')
        print('Saved generated music and reconstruction.')

reconstruct_multiple()

(20124,)
(20124,)
(20124,)
(20124,)
(20124,)
(20124,)
(20124,)
(20124,)
RBM reconstruction: 17472/17628. Time (m:s) - 41:3
Saved generated music and reconstruction.
(20124,)
(20124,)
(20124,)
(20124,)
(20124,)
(20124,)
(20124,)
(20124,)
RBM reconstruction: 17472/17628. Time (m:s) - 41:14
Saved generated music and reconstruction.
(20124,)
(20124,)
(20124,)
(20124,)
(20124,)
(20124,)
(20124,)
(20124,)
RBM reconstruction: 17472/17628. Time (m:s) - 41:25
Saved generated music and reconstruction.
(20124,)
(20124,)
(20124,)
(20124,)
(20124,)
(20124,)
(20124,)
(20124,)
RBM reconstruction: 17472/17628. Time (m:s) - 41:37
Saved generated music and reconstruction.
(20124,)
(20124,)
(20124,)
(20124,)
(20124,)
(20124,)
(20124,)
(20124,)
RBM reconstruction: 17472/17628. Time (m:s) - 41:48
Saved generated music and reconstruction.
(20124,)
(20124,)
(20124,)
(20124,)
(20124,)
(20124,)
(20124,)
(20124,)
RBM reconstruction: 17472/17628. Time (m:s) - 42:0
Saved generated music and reconstruction.
(20124